<a href="https://colab.research.google.com/github/ScriptSherpa/Web-scrapping/blob/main/webscraping_Indeed_ipynb_ii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip3 install wordcloud # (if you don't have already)
import pandas as pd
import numpy as np
import PIL              # for images
import wordcloud        # ploting word cloud
import requests         # grab web-page
import pickle           # save file
from bs4 import BeautifulSoup as bsopa  # parse web-page
import datetime         # format date/time
from collections import defaultdict
import re               # regular expressions

In [ ]:
# if you don't have wordcloud, get that now:

!pip3 install wordcloud
# conda install -c conda-forge wordcloud

In [ ]:
'''
ex.) https://www.indeed.com/jobs?q=data+scientist&l=california&start=10

range(0,150,10): each page will have "start=0,start=10,start=20 etc" which deals with
going through each page, but not exactly 10 entries/pg.

string formatting is used to denote what job and location we want: you can use a string
separated by space and it will be interpreted by the website.

sou=bsopa(y.text,'lxml') is taking our get request and converting to text in the format
of 'lxml' but we can replace with 'html.parser' as well.

Each job post can be parsed by the 'div',{"class":"jobsearch-SerpJobCard"} or
'div', {'class': 'row'} depending on how you want to search

After that we get each piece of information we want to obtain: job title, location etc.

the only difficult and frustrating part is getting all the raw text for each posting
relating to the qulifications. We have to open a link, iterate through it and then extract
all the information with a try/except block and then further process.
'''

gg=[]
for j in range(0,150,10):
    position,location='data scientist','california'
    y=requests.get('https://www.indeed.com/jobs?q={}&l={}&sort=date='.format(position,location)+str(j))

    # y=requests.get('https://www.indeed.com/jobs?q=data+scientist&l=california&sort=date='+str(i))
    sou=bsopa(y.text,'lxml')

#     for ii in sou.find_all('div', {'class': 'row'}):
    for ii in sou.find_all('div',{"class":"jobsearch-SerpJobCard"}):

        job_title = ii.find('a', {'data-tn-element': 'jobTitle'})['title']
        company_name = ii.find('span', {'class': 'company'}).text.strip()
        location=ii.find('span',{"class":"location"})
        post_date = ii.find('span', attrs={'class': 'date'})
        summary=ii.find('div',attrs={'class':'summary'})

        if location:
            location=location.text.strip()
        else:
            location=ii.find('div',{"class":"location"})
            location=location.text.strip()

        k=ii.find('h2', {'class':"title"})
        p=k.find(href=True)
        v=p['href']
        f_=str(v).replace('&amp;','&') # links to iterate for qualification text


        datum = {'job_title': job_title,
                    'company_name': company_name,
                    'location': location,
                    'summary':summary.text.strip(),
                    'post_Date':post_date.text,
                    'Qualification_link': f_}

#         gg.append([location,job_title,company_name,post_date.text,summary.text.strip()
#                   ,f_])
        gg.append(datum)

In [ ]:
len(gg)
gg[0]

IndexError: list index out of range

In [ ]:
# get all qualification page text: key=index, value=string of text for qualification
hoop=[]
for i in range(len(gg)):
    op=requests.get('https://www.indeed.com'+gg[i]['Qualification_link'])
    sou_=bsopa(op.text,'html.parser')
    for ii in sou_.find('div',{'class':'jobsearch-jobDescriptionText'}):
        try:
            hoop.append([i,''.join(ii.text.strip())])
        except AttributeError:
            hoop.append([i,''])
hoop[0]

IndexError: list index out of range

In [ ]:
# create dictionary with values as lists
dct_lst= defaultdict(list)
for i in hoop:
    dct_lst[i[0]].append(i[1])
u=[]
for i in dct_lst.values(): # string join: lists of lists of strings
    u.append(''.join(i))

In [ ]:
# one entry of our qualification text:
u[2]

IndexError: list index out of range

In [ ]:
jobs_=pd.concat([pd.DataFrame(gg),pd.DataFrame(u,columns=['Qual_Text'])],axis=1)
jobs_.head()

In [ ]:
import re
v=[]
for i in jobs_['post_Date']:

    if re.findall(r'[0-9]',i):
        # if the string has digits convert each entry to single string: ['3','0']->'30'
        b=''.join(re.findall(r'[0-9]',i))

        # convert string int to int and subtract from today's date and format
        g=(datetime.datetime.today()-datetime.timedelta(int(b))).strftime('%m-%d-%Y')

        v.append(g)

    else: # this will contain strings like: 'just posted' or 'today' etc before convert
        v.append(datetime.datetime.today().strftime('%m-%d-%Y'))
v[:5]


In [ ]:

# fixed posting date to date format instead of string: last column
jobs_['posting_date_fixed']
jobs_.head()

NameError: name 'jobs_' is not defined

In [ ]:

buzz_words=['Python','SQL','AWS', 'Machine learning','Deep learning','Text mining',
'NLP','SAS','Tableau','Sagemaker','Tensorflow','Spark', 'numpy', 'MongDB','PSQL',
"Postgres", 'Pandas', 'RESTFUL','NLP','Statistics','Algorithms','Visualization',
'GCP','Google Cloud','Naive Bayes','Random Forest','Bachelors degree','Masters degree'
'Java','Pyspark','Postgres','MySQL','Github','Docker','Machine Learning','C+',
'C++','Pytorch','Jupyter Notebook','R Studio','R-Studio','Forecasting','Hive',
'PhD','GCP','Numpy','NoSQL','Neo4j','Neural Network','Clustering','Linear Algebra',
'Google Colab','Data Mining','Regression','Time Series','ETL','Data Wrangling',
'Web Scraping','Feature Extraction','Featuring Engineering','Scipy','ML','DL']
buzz_words_list=[x.lower() for x in buzz_words]  # convert list to lowercase to parse

yo=[]
for i in range(len(jobs_.Qual_Text)):
    a=buzz_words_list
    dd=[x for x in a if x in jobs_.Qual_Text[i].lower()]
    yo.append(dd)
jobs_['skill_matches']=yo
jobs_.head(7)

NameError: name 'jobs_' is not defined

In [ ]:
filename='Indeed_scrape_Oct2020.txt'
file=open(filename,'wb')
pickle.dump(jobs_,file)

file_ =open(filename,'rb')
new_file_ =pickle.load(file_)
new_file_.head(10)

NameError: name 'jobs_' is not defined